In [1]:
import xarray as xr
import numpy as np
import os


In [55]:
print('import sonde data')

inputdir = '/Users/bfildier/Data/EUREC4A/merged/sondes'
radiosondes = xr.open_dataset(os.path.join(inputdir,'all_radiosondes.nc'))
dropsondes = xr.open_dataset(os.path.join(inputdir,'all_dropsondes.nc'))

import sonde data


In [59]:
radiosondes.reset_coords(['lon','lat','time'],drop=True)
radiosondes.coords

Coordinates:
  * gpsalt       (gpsalt) float32 0.0 10.0 20.0 30.0 ... 29970.0 29980.0 29990.0
    time         (launch_time, gpsalt) datetime64[ns] ...
  * launch_time  (launch_time) datetime64[ns] 2020-01-21T18:58:35 ... 2020-02-12T14:44:31
    lat          (launch_time, gpsalt) float32 ...
    lon          (launch_time, gpsalt) float32 ...

In [58]:
# set(radiosondes.platform.values)

# drop lon,lat coordinates in radiosondes
# del radiosondes.coords['lat']
# del radiosondes.coords['lon']
# del radiosondes.coords['time']
radiosondes.reset_coords(['lon','lat','time'],drop=True)
# radiosondes.drop('lat')
# radiosondes.drop('lon')
# radiosondes.drop('time')

# del dropsondes['trajectory'] # only has empty values

# merge
all_sondes = xr.concat([dropsondes,radiosondes],dim='launch_time')

ValueError: {'time', 'lat', 'lon'} is a coordinate in some datasets but not others.

In [37]:
radiosondes.time

AttributeError: 'Dataset' object has no attribute 'time'